# LLM Iteration 27

**Aim**

Correct the use of random seeds from iteration 26 so that the different replications have different results.

**Prompt**

The use of a single random seed in the scenario class means that all of the replications have the same output. Each time the warmup_analysis function is run, the same output appears due to the random seeds being used. Use the scenario.generate_seeds method to correct this so each replication has different results.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**Additional code to run model**

In [1]:
def trace(msg):
    '''
    Utility function for printing simulation.
    Set the TRACE constant to FALSE to turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

In [2]:
TRACE = False

In [3]:
import simpy
import numpy as np
import math
import pandas as pd

In [4]:
class Scenario:
    def __init__(self, num_beds, run_length=60*2, seed=None):
        self.num_beds = num_beds
        self.run_length = run_length
        self.seed = seed
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self, num_runs):
        interarrival_distributions = {}
        seeds = self.generate_seeds(num_runs)
        for patient_type, seed in seeds.items():
            if seed[0] is not None:
                np.random.seed(seed)
                interarrival_distributions[patient_type] = np.random.\
                    default_rng(seed)
            else:
                interarrival_distributions[patient_type] = np.random.\
                    default_rng()
        return interarrival_distributions

    def generate_seeds(self, num_runs):
        seeds = {}
        for patient_type, seed in self.random_seeds.items():
            if self.seed is not None:
                seeds[patient_type] = [seed + i + self.seed for i in \
                                       range(num_runs)]
            else:
                seeds[patient_type] = [None for i in range(num_runs)]
        return seeds

In [5]:
class StrokeWard:
    def __init__(self, scenario):
        self.env = simpy.Environment()
        self.beds = simpy.Resource(self.env, capacity=scenario.num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        self.interarrival_distributions = scenario.\
            create_distributions(num_runs=1)
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.patient_type_store = []
        self.los_store = []

    def update_length_of_stay(self, patient_type, destination):
        if patient_type == 'Stroke':
            if destination == 'ESD':
                return {"mean": 4.6, "std_dev": 4.8}
            elif destination == 'Other':
                # Mortality rate for Stroke patients going to Other
                if np.random.rand() < 0.02:
                     # Adjusted parameters for mortality case
                    return {"mean": 7.0, "std_dev": 8.7}
        return self.length_of_stay[patient_type]

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))

            trace(f'Patient {patient.id} ({patient.patient_type}) admitted '
                  f'to the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())

            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)

            length_of_stay_params = self.update_length_of_stay(patient.\
                                                               patient_type,
                                                               route_choice)

            mu, sigma = calculate_mu_sigma(length_of_stay_params["mean"],
                                           length_of_stay_params["std_dev"])

            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))

            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))

            trace(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            trace(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].exponential(
                        scenario.default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)

        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }

        summary_df = pd.DataFrame(summary_data)

        print("Summary:")
        print(summary_df)

In [6]:
class WarmupAuditor:
    def __init__(self, scenario, audit_interval):
        self.scenario = scenario
        self.audit_interval = audit_interval
        self.audit_times = []
        self.waiting_patients_audit = []
        self.wait_times_audit = []

    def run_model(self):
        stroke_ward = StrokeWard(self.scenario)
        env = stroke_ward.env

        PATIENTS = ["Stroke", "TIA", "Complex Neurological", "Other"]
        for patient_type in PATIENTS:
            env.process(stroke_ward.patient_arrival(patient_type))

        env.process(self.audit_model(stroke_ward))
        env.run(until=self.scenario.run_length)

        return self.summary()

    def audit_model(self, stroke_ward):
        while stroke_ward.env.now < self.scenario.run_length:
            yield stroke_ward.env.timeout(self.audit_interval)
            self.audit_times.append(stroke_ward.env.now)
            self.waiting_patients_audit.append(np.mean(stroke_ward.\
                                                       waiting_patients))
            self.wait_times_audit.append(np.mean(stroke_ward.wait_times))

    def summary(self):
        audit_data = {
            'Audit Time': self.audit_times,
            'Mean Waiting Patients': self.waiting_patients_audit,
            'Mean Wait Time': self.wait_times_audit
        }

        summary_df = pd.DataFrame(audit_data)

        return summary_df

In [7]:
def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma

class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

**LLM Output**

To address this issue and ensure that each replication produces different results, we need to generate unique random seeds for each replication.
The scenario.generate_seeds method can be utilized to generate a set of unique random seeds for each replication. By modifying the warmup_analysis function to create a new Scenario instance with updated random seeds for each replication, we can ensure that each replication uses a different set of random seeds, resulting in different outputs.
Here's the updated warmup_analysis function that incorporates this change:

In [8]:
def warmup_analysis(scenario, num_replications, audit_interval):
    waiting_patients_results = {}
    wait_times_results = {}

    for i in range(num_replications):
        ### MODIFICATION - added scenario.seed so None can be used ###
        scenario_with_seeds = Scenario(scenario.num_beds, scenario.run_length,
                                       seed=scenario.seed)
        scenario_with_seeds.random_seeds = {k: v[i] for k, v in scenario.\
                                            generate_seeds(num_replications).\
                                            items()}

        warmup_auditor = WarmupAuditor(scenario_with_seeds, audit_interval)
        audit_results = warmup_auditor.run_model()

        waiting_patients_results[f'Replication {i+1}'] = audit_results\
            ['Mean Waiting Patients']
        wait_times_results[f'Replication {i+1}'] = audit_results\
            ['Mean Wait Time']

    waiting_patients_df = pd.DataFrame(waiting_patients_results)
    wait_times_df = pd.DataFrame(wait_times_results)

    waiting_patients_df = waiting_patients_df.set_index(audit_results\
                                                        ['Audit Time'])
    wait_times_df = wait_times_df.set_index(audit_results['Audit Time'])

    results = {
        'Waiting Patients': waiting_patients_df,
        'Wait Times': wait_times_df
    }

    return results

In this updated warmup_analysis function:
- A new instance of the Scenario class is created for each replication, with the same num_beds and run_length as the original scenario.
- The random_seeds attribute of the new Scenario instance is updated with a set of unique random seeds generated using the scenario.generate_seeds(num_replications) method.
- The WarmupAuditor instance is created with the new Scenario instance, ensuring that each replication uses a different set of random seeds.
- The rest of the function remains the same, collecting and storing the audit results for each replication in the respective DataFrames.

By creating a new Scenario instance with updated random seeds for each replication, the warmup_analysis function ensures that each replication uses a different set of random seeds, resulting in different outputs across replications. This approach addresses the issue of obtaining the same output due to the use of a single random seed across all replications.

**Code to run model**

In [9]:
scenario = Scenario(num_beds=10, seed=42)
num_replications = 5
audit_interval = 5

warmup_results = warmup_analysis(scenario, num_replications, audit_interval)

print("Waiting Patients:")
print(warmup_results['Waiting Patients'])
print("\nWait Times:")
print(warmup_results['Wait Times'])

Waiting Patients:
            Replication 1  Replication 2  Replication 3  Replication 4  \
Audit Time                                                               
5                0.000000       0.000000       0.000000       0.000000   
10               0.000000       0.000000       0.000000       0.050000   
15               0.000000       0.000000       0.000000       0.161290   
20               0.000000       0.000000       0.277778       0.181818   
25               0.000000       0.000000       0.272727       0.139535   
30               0.000000       0.018519       0.288462       0.122449   
35               0.000000       0.016129       0.267857       0.111111   
40               0.000000       0.229730       0.241935       0.101695   
45               0.000000       0.215190       0.211268       0.086957   
50               0.050633       0.202381       0.205128       0.083333   
55               0.604651       0.184783       0.195122       0.076923   
60               1.5

**Checks**

In [10]:
# Using no random seed
scenario = Scenario(num_beds=10, seed=None)
num_replications = 5
audit_interval = 5

warmup_results = warmup_analysis(scenario, num_replications, audit_interval)

print("Waiting Patients:")
print(warmup_results['Waiting Patients'])
print("\nWait Times:")
print(warmup_results['Wait Times'])

Waiting Patients:
            Replication 1  Replication 2  Replication 3  Replication 4  \
Audit Time                                                               
5                0.000000       0.000000       0.000000       0.000000   
10               0.000000       0.000000       0.000000       0.000000   
15               0.000000       0.000000       0.000000       0.000000   
20               0.000000       0.000000       0.000000       0.000000   
25               0.000000       0.000000       0.048780       0.000000   
30               0.000000       0.000000       0.040816       0.000000   
35               0.000000       0.053571       0.034483       0.019231   
40               0.000000       0.046875       0.062500       0.033333   
45               0.000000       0.041667       0.101449       0.164179   
50               0.000000       0.040541       0.092105       0.243243   
55               0.000000       0.037500       0.088608       0.280000   
60               0.0

In [11]:
# Changing number of replications
scenario = Scenario(num_beds=10, seed=42)
num_replications = 8
audit_interval = 5

warmup_results = warmup_analysis(scenario, num_replications, audit_interval)

print("Waiting Patients:")
print(warmup_results['Waiting Patients'])
print("\nWait Times:")
print(warmup_results['Wait Times'])

Waiting Patients:
            Replication 1  Replication 2  Replication 3  Replication 4  \
Audit Time                                                               
5                0.000000       0.000000       0.000000       0.000000   
10               0.000000       0.000000       0.000000       0.050000   
15               0.000000       0.000000       0.000000       0.161290   
20               0.000000       0.000000       0.277778       0.181818   
25               0.000000       0.000000       0.272727       0.139535   
30               0.000000       0.018519       0.288462       0.122449   
35               0.000000       0.016129       0.267857       0.111111   
40               0.000000       0.229730       0.241935       0.101695   
45               0.000000       0.215190       0.211268       0.086957   
50               0.050633       0.202381       0.205128       0.083333   
55               0.604651       0.184783       0.195122       0.076923   
60               1.5

In [12]:
# Changing number of beds
scenario = Scenario(num_beds=1, seed=42)
num_replications = 4
audit_interval = 5

warmup_results = warmup_analysis(scenario, num_replications, audit_interval)

print("Waiting Patients:")
print(warmup_results['Waiting Patients'])
print("\nWait Times:")
print(warmup_results['Wait Times'])

Waiting Patients:
            Replication 1  Replication 2  Replication 3  Replication 4
Audit Time                                                            
5                0.000000       0.000000       4.000000       1.500000
10               3.500000       0.000000       4.000000       4.333333
15               3.500000       0.000000       7.333333       4.333333
20               3.500000      14.000000      11.750000      10.000000
25              11.000000      14.000000      11.750000      13.000000
30              17.750000      24.000000      16.400000      13.000000
35              17.750000      30.500000      27.250000      22.666667
40              17.750000      30.500000      27.250000      25.000000
45              17.750000      36.200000      33.400000      29.583333
50              51.363636      41.333333      38.166667      31.538462
55              54.846154      41.333333      38.166667      33.571429
60              54.846154      47.571429      43.642857    

In [13]:
# Changing audit interval
scenario = Scenario(num_beds=10, seed=42)
num_replications = 4
audit_interval = 10

warmup_results = warmup_analysis(scenario, num_replications, audit_interval)

print("Waiting Patients:")
print(warmup_results['Waiting Patients'])
print("\nWait Times:")
print(warmup_results['Wait Times'])

Waiting Patients:
            Replication 1  Replication 2  Replication 3  Replication 4
Audit Time                                                            
10               0.000000       0.000000       0.000000       0.050000
20               0.000000       0.000000       0.277778       0.181818
30               0.000000       0.018519       0.288462       0.122449
40               0.000000       0.229730       0.241935       0.101695
50               0.050633       0.202381       0.205128       0.083333
60               1.580000       0.210000       0.211111       0.071429
70               2.034188       0.829060       0.188119       0.305263
80               1.775362       0.801587       0.161017       0.544643
90               1.632258       0.711268       0.143939       0.491935
100              1.497041       0.664474       0.142857       0.432624
110              1.382514       0.601190       0.130178       0.396104

Wait Times:
            Replication 1  Replication 2  Repl

**Reflection**

The LLM has corrected the error and now the results are repeatable but with different results for each replications using the same implementation as used for multiple_replications. It removed the option for using no random seeds, but this was easily fixable by adding the scenario.seeds to the Scenario class in each replication. The audit interval can be changed to collect results at different time points. Changing the number of beds to 1 increased waiting times and number of patients waiting as requested. The number of replications can also be changed. 